In [1]:
from suicide_squad.DataUtils import *
from suicide_squad.DataProcessing import *

In [2]:
squad_to_csv(files,download_folder)

Converting Files to CSV
Finished Downloading and Converting to CSV


['D:\\GitHub\\Suicide-SQuAD\\data\\original\\train-v1.1.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\dev-v1.1.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\train-v2.0.csv',
 'D:\\GitHub\\Suicide-SQuAD\\data\\original\\dev-v2.0.csv']

In [3]:
dfs=squad_to_df(files,download_folder)

Converting Files to CSV
Finished Downloading and Converting to CSV
Converting Files to DataFrame
Finished Downloading and Converting to DataFrame


In [4]:
dfs.keys()

dict_keys(['train-v1.1', 'dev-v1.1', 'train-v2.0', 'dev-v2.0'])

In [5]:
examples=dfs['train-v1.1'].dropna(subset=['answer_text'])

In [6]:
examples.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,5733be284776f41900661182,False
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,5733be284776f4190066117f,False
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,5733be284776f41900661180,False
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,5733be284776f41900661181,False
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,5733be284776f4190066117e,False


In [7]:
from transformers import BertTokenizer, BertForQuestionAnswering,BertConfig
from transformers.tokenization_bert import BasicTokenizer, whitespace_tokenize


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
def do(target):
    text_id=' '.join([str(item) for item in tokenizer.encode(target.question_text,target.doc_text,max_length=512)[1:-1]])
    answer_id=' '.join([str(item) for item in tokenizer.encode(target.answer_text,max_length=512)[1:-1]])
    if((target.answer_text) and (target.question_text) and (target.doc_text)):
        try:
            matches=[match for match in re.finditer(answer_id, text_id)]
            start=matches[0].start()
            end=matches[0].end()
            target['start_label']=len(text_id[:start].split(' '))
            target['end_label']=len(answer_id.split(' '))+len(text_id[:start].split(' '))
        except:
            target['start_label']=0
            target['end_label']=0
        target['text']='[CLS] '+target['question_text']+' [SEP] '+target['doc_text']+' [SEP]'
    return target

out=examples.apply(lambda x:do(x),axis=1)

In [9]:
# out.to_csv(os.path.join(target_folder,'dev_v1_1.csv'))

In [71]:
# with torch.no_grad():
#     for data_item in test_dataloader:
#         bert_model_output: Tuple[torch.Tensor, torch.Tensor] = bert_qa_module(input_ids = data_item[0], token_type_ids =
#         data_item[1], attention_mask = data_item[4], output_all_encoded_layers = False)

#         answer_start_index_batch_as_matrix = data_item[2].unsqueeze(dim = 1).to(device = device)
#         answer_end_index_batch_as_matrix = data_item[3].unsqueeze(dim = 1).to(device = device)

#         answer_start_index_loss = loss_function(bert_model_output[0], answer_start_index_batch_as_matrix)
#         answer_end_index_loss = loss_function(bert_model_output[1], answer_end_index_batch_as_matrix)
#         total_loss = answer_start_index_loss + answer_end_index_loss